In [1]:
import pandas as pd
import numpy as np
from numpy import concatenate
from pandas import concat
from pandas import read_csv
from pandas import DataFrame
import tensorflow as tf
from tensorflow import keras
import os
import datetime
import IPython
import IPython.display
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import explained_variance_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Flatten, LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Bidirectional, Input, Reshape, RepeatVector, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
%matplotlib inline
from scipy.stats import randint as sp_randint
np.random.seed(123) # for reproducibility
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier, KerasRegressor
import keras_tuner

In [2]:
# CARGAR LA BASE DE DATOS
from pandas import read_csv
datos = read_csv('Giron_Lags.csv', encoding='latin-1', sep=";")
# Dropping the id and date columns
datos.columns

Index(['Municipio', 'Año', 'Semana_Epi', 'Casos_Dengue',
       'Temp_Seca_Max_Dia_Prom', 'Temp_Seca_Max_Dia_Prom_1',
       'Temp_Seca_Max_Dia_Prom_2', 'Temp_Seca_Max_Dia_Prom_3',
       'Temp_Seca_Max_Dia_Prom_4', 'Temp_Seca_Max_Dia_Prom_5',
       'Temp_Seca_Max_Dia_Prom_6', 'Temp_Seca_Max_Sem', 'Temp_Seca_Max_Sem_1',
       'Temp_Seca_Max_Sem_2', 'Temp_Seca_Max_Sem_3', 'Temp_Seca_Max_Sem_4',
       'Temp_Seca_Max_Sem_5', 'Temp_Seca_Max_Sem_6', 'Temp_Seca_Min_Dia_Prom',
       'Temp_Seca_Min_Dia_Prom_1', 'Temp_Seca_Min_Dia_Prom_2',
       'Temp_Seca_Min_Dia_Prom_3', 'Temp_Seca_Min_Dia_Prom_4',
       'Temp_Seca_Min_Dia_Prom_5', 'Temp_Seca_Min_Dia_Prom_6',
       'Temp_Seca_Min_Sem', 'Temp_Seca_Min_Sem_1', 'Temp_Seca_Min_Sem_2',
       'Temp_Seca_Min_Sem_3', 'Temp_Seca_Min_Sem_4', 'Temp_Seca_Min_Sem_5',
       'Temp_Seca_Min_Sem_6', 'Prom_Dia_Preci', 'Prom_Dia_Preci_1',
       'Prom_Dia_Preci_2', 'Prom_Dia_Preci_3', 'Prom_Dia_Preci_4',
       'Prom_Dia_Preci_5', 'Prom_Dia_Preci_6', '

In [3]:
datos = datos.drop(['Municipio', 'Año', 'Semana_Epi', 'Temp_Seca_Max_Dia_Prom_1',
       'Temp_Seca_Max_Dia_Prom_2','Temp_Seca_Max_Dia_Prom_3','Temp_Seca_Max_Dia_Prom_4', 
       'Temp_Seca_Max_Dia_Prom_5','Temp_Seca_Max_Dia_Prom_6', 'Temp_Seca_Max_Sem_1',
       'Temp_Seca_Max_Sem_2', 'Temp_Seca_Max_Sem_3', 'Temp_Seca_Max_Sem_4',
       'Temp_Seca_Max_Sem_5', 'Temp_Seca_Max_Sem_6','Temp_Seca_Min_Dia_Prom_1',
       'Temp_Seca_Min_Dia_Prom_2','Temp_Seca_Min_Dia_Prom_3', 'Temp_Seca_Min_Dia_Prom_4',
       'Temp_Seca_Min_Dia_Prom_5', 'Temp_Seca_Min_Dia_Prom_6','Temp_Seca_Min_Sem_1',
       'Temp_Seca_Min_Sem_2','Temp_Seca_Min_Sem_3', 'Temp_Seca_Min_Sem_4', 
       'Temp_Seca_Min_Sem_5','Temp_Seca_Min_Sem_6','Prom_Dia_Preci_1',
       'Prom_Dia_Preci_2', 'Prom_Dia_Preci_3', 'Prom_Dia_Preci_4',
       'Prom_Dia_Preci_5', 'Prom_Dia_Preci_6','Total_Preci_Sem_1',
       'Total_Preci_Sem_2', 'Total_Preci_Sem_3','Total_Preci_Sem_4',
       'Total_Preci_Sem_5', 'Total_Preci_Sem_6','Temp_Max_Dia_Prom_1',
       'Temp_Max_Dia_Prom_2','Temp_Max_Dia_Prom_3', 'Temp_Max_Dia_Prom_4',
       'Temp_Max_Dia_Prom_5','Temp_Max_Dia_Prom_6','Temp_Max_Sem_1','Temp_Max_Sem_2',
       'Temp_Max_Sem_3', 'Temp_Max_Sem_4', 'Temp_Max_Sem_5','Temp_Max_Sem_6',
       'Temp_Min_Dia_Prom_1','Temp_Min_Dia_Prom_2', 'Temp_Min_Dia_Prom_3',
       'Temp_Min_Dia_Prom_4','Temp_Min_Dia_Prom_5', 'Temp_Min_Dia_Prom_6',
       'Temp_Min_Sem_1', 'Temp_Min_Sem_2', 'Temp_Min_Sem_3', 'Temp_Min_Sem_4',
       'Temp_Min_Sem_5', 'Temp_Min_Sem_6','Hum_Rel_Max_1','Hum_Rel_Max_2',
       'Hum_Rel_Max_3', 'Hum_Rel_Max_4', 'Hum_Rel_Max_5','Hum_Rel_Max_6',
       'Hum_Rel_Min_1', 'Hum_Rel_Min_2','Hum_Rel_Min_3', 'Hum_Rel_Min_4',
       'Hum_Rel_Min_5', 'Hum_Rel_Min_6'],axis=1)
datos.columns

Index(['Casos_Dengue', 'Temp_Seca_Max_Dia_Prom', 'Temp_Seca_Max_Sem',
       'Temp_Seca_Min_Dia_Prom', 'Temp_Seca_Min_Sem', 'Prom_Dia_Preci',
       'Total_Preci_Sem', 'Temp_Max_Dia_Prom', 'Temp_Max_Sem',
       'Temp_Min_Dia_Prom', 'Temp_Min_Sem', 'Hum_Rel_Max', 'Hum_Rel_Min'],
      dtype='object')

In [4]:
from sklearn.impute import KNNImputer
knnImputer = KNNImputer (missing_values=np.nan, n_neighbors=2, 
                         weights="uniform", metric="nan_euclidean")
# Ajustamos el modelo e imputamos los missing values

knnImputer.fit(datos[['Temp_Seca_Max_Dia_Prom']])
datos['Temp_Seca_Max_Dia_Prom'] = knnImputer.transform(datos[['Temp_Seca_Max_Dia_Prom']]).ravel()
knnImputer.fit(datos[['Temp_Seca_Max_Sem']])
datos['Temp_Seca_Max_Sem'] = knnImputer.transform(datos[['Temp_Seca_Max_Sem']]).ravel()
knnImputer.fit(datos[['Temp_Seca_Min_Dia_Prom']])
datos['Temp_Seca_Min_Dia_Prom'] = knnImputer.transform(datos[['Temp_Seca_Min_Dia_Prom']]).ravel()
knnImputer.fit(datos[['Temp_Seca_Min_Sem']])
datos['Temp_Seca_Min_Sem'] = knnImputer.transform(datos[['Temp_Seca_Min_Sem']]).ravel()
knnImputer.fit(datos[['Prom_Dia_Preci']])
datos['Prom_Dia_Preci'] = knnImputer.transform(datos[['Prom_Dia_Preci']]).ravel()
knnImputer.fit(datos[['Total_Preci_Sem']])
datos['Total_Preci_Sem'] = knnImputer.transform(datos[['Total_Preci_Sem']]).ravel()
knnImputer.fit(datos[['Temp_Max_Dia_Prom']])
datos['Temp_Max_Dia_Prom'] = knnImputer.transform(datos[['Temp_Max_Dia_Prom']]).ravel()
knnImputer.fit(datos[['Temp_Max_Sem']])
datos['Temp_Max_Sem'] = knnImputer.transform(datos[['Temp_Max_Sem']]).ravel()
knnImputer.fit(datos[['Temp_Min_Dia_Prom']])
datos['Temp_Min_Dia_Prom'] = knnImputer.transform(datos[['Temp_Min_Dia_Prom']]).ravel()
knnImputer.fit(datos[['Temp_Min_Sem']])
datos['Temp_Min_Sem'] = knnImputer.transform(datos[['Temp_Min_Sem']]).ravel()
knnImputer.fit(datos[['Hum_Rel_Max']])
datos['Hum_Rel_Max'] = knnImputer.transform(datos[['Hum_Rel_Max']]).ravel()
knnImputer.fit(datos[['Hum_Rel_Min']])
datos['Hum_Rel_Min'] = knnImputer.transform(datos[['Hum_Rel_Min']]).ravel()


datos.shape

(364, 13)

In [5]:
train_size = int(len(datos) * 0.8)
test_size = len(datos) - train_size
train, test = datos.iloc[0:train_size], datos.iloc[train_size:len(datos)]
print(len(train), len(test))

291 73


In [6]:
f_columns = ['Temp_Seca_Max_Dia_Prom', 'Temp_Seca_Max_Sem', 'Temp_Seca_Min_Dia_Prom', 'Temp_Seca_Min_Sem', 'Prom_Dia_Preci',
             'Total_Preci_Sem', 'Temp_Max_Dia_Prom', 'Temp_Max_Sem',
             'Temp_Min_Dia_Prom', 'Temp_Min_Sem', 'Hum_Rel_Max', 'Hum_Rel_Min']
f_transformer = MinMaxScaler(feature_range=(0, 1))
f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())

In [7]:
dengue_transformer = MinMaxScaler(feature_range=(0, 1))
dengue_transformer = dengue_transformer.fit(train[['Casos_Dengue']])
train['Casos_Dengue'] = dengue_transformer.transform(train[['Casos_Dengue']])
test['Casos_Dengue'] = dengue_transformer.transform(test[['Casos_Dengue']])

In [8]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [9]:
time_steps = 7
# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(train, train.Casos_Dengue, time_steps)
X_test, y_test = create_dataset(test, test.Casos_Dengue, time_steps)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape )

(284, 7, 13) (284,) (66, 7, 13) (66,)


In [10]:
def cnn_model(dropout_rate=0.2, optimizer="adam", activation="relu", No_of_CONV_and_Maxpool_layers=3,
              No_of_Units_in_dense_layers=100, padding ='same', Conv1d_filters=60):

    
    #model sequential
    cnn_model=Sequential()
    
    #CAPA DE ENTRADA
    cnn_model.add(keras.layers.Conv1D(32, kernel_size=2, strides=1,
                                      activation=activation, input_shape=(X_train.shape[1], X_train.shape[2]),padding=padding))
    
    
    for i in range(No_of_CONV_and_Maxpool_layers):
        
        cnn_model.add(keras.layers.Conv1D(Conv1d_filters, kernel_size=2, strides=1, activation=activation, padding=padding))
        cnn_model.add(keras.layers.MaxPooling1D(pool_size=2, padding=padding))
        
    cnn_model.add(Dropout(dropout_rate))
    cnn_model.add(Flatten())
    
    
    
    cnn_model.add(keras.layers.Dense(units=No_of_Units_in_dense_layers, activation=activation))
    cnn_model.add(Dropout(dropout_rate)) 
                  
    cnn_model.add(Dense(1))
    
    
    #COMPILACIÓN DEL MODELO
    cnn_model.compile(optimizer=optimizer, loss = 'mse', metrics = ['mse'])
    
    

    return cnn_model

In [11]:
#HIPERPARÁMETROS

activation=['relu','tanh', 'sigmoid', 'linear']
dropout_rate=[0.1, 0.3, 0.5]
No_of_CONV_and_Maxpool_layers=[2,3,4]
Conv1d_filters=[60,40,80,120,220]
No_of_Units_in_dense_layers=[128,64,32,256]
optimizer = ['adam', 'sgd', 'rmsprop']
padding = ['valid', 'same', 'causal']

param_grid = dict(model__activation=activation,
                  model__dropout_rate=dropout_rate,
                  model__padding=padding,
                  model__No_of_CONV_and_Maxpool_layers=No_of_CONV_and_Maxpool_layers,
                  model__Conv1d_filters=Conv1d_filters,
                  model__No_of_Units_in_dense_layers=No_of_Units_in_dense_layers,
                  model__optimizer=optimizer,)

In [12]:
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer

model =  KerasRegressor(model=cnn_model, verbose=0)

rs = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring='neg_mean_squared_error',
                        n_jobs=-1, cv=KFold(3), n_iter=20, verbose=0,random_state = 123, return_train_score = True)

In [13]:
fitted_model = rs.fit(X_train, y_train)

In [14]:
resultados = pd.DataFrame(fitted_model.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False)\
    .head(10)

,param_model__padding,param_model__optimizer,param_model__dropout_rate,param_model__activation,param_model__No_of_Units_in_dense_layers,param_model__No_of_CONV_and_Maxpool_layers,param_model__Conv1d_filters,mean_test_score,std_test_score,mean_train_score,std_train_score
4,same,rmsprop,0.1,tanh,128,2,220,-0.053546,0.026052,-0.034836,0.017534
3,same,adam,0.3,sigmoid,64,2,120,-0.058003,0.038734,-0.192013,0.094559
1,same,rmsprop,0.5,linear,256,3,80,-0.067798,0.068488,-0.032393,0.012363
6,same,sgd,0.3,linear,64,4,220,-0.081906,0.071188,-0.038553,0.014184
18,same,rmsprop,0.5,relu,128,4,80,-0.084052,0.058915,-0.043733,0.019309
9,same,rmsprop,0.5,relu,32,3,220,-0.084877,0.095038,-0.050488,0.019460
19,same,sgd,0.3,sigmoid,128,2,40,-0.096947,0.078989,-0.051295,0.020648
0,valid,adam,0.5,relu,64,4,80,NaN,NaN,NaN,NaN
2,causal,sgd,0.5,tanh,128,2,40,NaN,NaN,NaN,NaN
5,valid,sgd,0.1,tanh,32,3,220,NaN,NaN,NaN,NaN


In [15]:
print("Best: %f using %s" % (fitted_model.best_score_, fitted_model.best_params_))

Best: -0.053546 using {'model__padding': 'same', 'model__optimizer': 'rmsprop', 'model__dropout_rate': 0.1, 'model__activation': 'tanh', 'model__No_of_Units_in_dense_layers': 128, 'model__No_of_CONV_and_Maxpool_layers': 2, 'model__Conv1d_filters': 220}
